In [8]:
from google import google
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy import MetaData
from configparser import ConfigParser
import os
import spacy
nlp = spacy.load('en')
import re
import phonenumbers
import contextlib

# from imwithdata.es_etl.issues_actions import (
#     issues,
#     actions,
#     state_regex,
#     city_regex,
#     web_url_regex,
#     date_include_regex,
#     date_exclude_regex,
#     leg_name_regex,
#     leg_twitter_regex
# )

In [2]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas','District of Columbia','California', 'Colorado', 
          'Connecticut', 'Delaware', 'Florida', 'Georgia', 
          'Hawaii', 'Idaho','Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
          'Massachusetts','Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
          'New Hampshire','New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 
          'Oregon','Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
          'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','ALABAMA', 'ALASKA', 'ARIZONA', 
          'ARKANSAS','DISTRICT OF COLUMBIA','CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 
          'HAWAII', 'IDAHO','ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 
          'MASSACHUSETTS','MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 
          'NEW HAMPSHIRE','NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 
          'OREGON','PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH',
          'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING','NY','NYC']

state_abbrevs = ['IA', 'KS', 'UT', 'MA','MI','MO',
          'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'Conn', 'NM', 'MS', 'GA',
          'CO', 'NJ', 'FL', 'MN', 'NV', 'AZ', 'WI', 'ND', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT',
          'NY', 'CA', 'HI', 'IL', 'TN', 'Mass', 'OH', 'MD', 'MI', 'WY', 'WA', 'SC','PA',
          'IN', 'LA', 'DC', 'AR', 'WV', 'TX','Minn','Ark','Ind','D\.C\.',
          'Mizz','Nev','Okl','Penn','S\. Carolina','N\. Carolina','S Carolina','N Carolina','Tenn','Tex']

state_regex = re.compile(r'(\\b' + '|\\b'.join(states) + '|' + '|'.join([state + '\\b' for state in states])
                                                    + '|' + '|'.join(['\\b' + state + '\\b' for state in state_abbrevs])+ r')')
print (state_regex)

re.compile('(\\\\bAlabama|\\bAlaska|\\bArizona|\\bArkansas|\\bDistrict of Columbia|\\bCalifornia|\\bColorado|\\bConnecticut|\\bDelaware|\\bFlorida|\\bGeorgia|\\bHawaii|\\bIdaho|\\bIllinois|\\bIndiana|\\bIowa|\\b)


In [3]:
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

mysql_creds = get_ini_vals(config_file, 'mysql')

engine = create_engine("""mysql+pymysql://{user}:{password}@{host}:{port}/{db}""".format(user=mysql_creds['user'],
                                                                                 password=mysql_creds['password'],
                                                                                 host=mysql_creds['host'],
                                                                                 port=mysql_creds['port'],
                                                                                 db=mysql_creds['database']
                                                                                )
                      )
conn = engine.connect()


In [4]:
final_data = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/final_data_example.csv')
# final_data.columns
final_data.drop(final_data.columns[0], axis=1,inplace=True)
# final_data.columns
actions = final_data.sort('total_score',ascending=[0])
actions.head()


issue_list = actions['issue'].tolist()
action_list = actions['action'].tolist()
id_list = actions['id'].tolist()

es_score_list = actions['es_score'].tolist()
total_score_list = actions['total_score'].tolist()
tweet_list = [tweet.capitalize() if tweet.isupper() else tweet for tweet in actions['tweet'].tolist() ]
tweet_timestamp_list = actions['tweet_timestamp'].tolist()
query_timestamp_list = actions['query_timestamp'].tolist()
user_list = actions['tweet_user'].tolist()


/Users/brosskatz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [5]:
print (len(final_data))

227


In [6]:
time_regex = re.compile(r'\d{1,2}(?:(?:am|pm)|(?::\d{1,2})(?:am|pm)?)', re.IGNORECASE)

dates = []
start_times = []
end_times = []
cities = []
states = []
legislators = []
legislator_handles = []
phone_numbers = []
titles = []
all_urls = []

for i, tweet in enumerate(tweet_list):
    date = ''
    start_time = ''
    end_time = ''
    city = ''
    state = ''
    legislator = ''
    legislator_handle = ''
    phone_number = ''
    url = ''
    title = ''
    
    doc = nlp(tweet)
    all_dates = [doc.text for doc in doc.ents if doc.label_ == 'DATE']
    date_matches = re.findall(date_include_regex, ' '.join(all_dates))
    ### EXCLUDE SOME DIRTY DATES FROM TWITTER THAT SPACY MISTAKENLY INCLUDES    
    if date_matches:
        date = all_dates[0]
    dates.append(date)
        
    times = re.findall(time_regex,tweet)
    if times:
        start_time = times[0]
    if len(times) > 1:
        end_time = times[1]
    start_times.append(start_time)
    end_times.append(end_time)
    
    tweet_cities = re.findall(city_regex,tweet)
    if tweet_cities:
        tweet_cities = list(set([city.title() for city in tweet_cities]))
        if len(tweet_cities) == 1:
            city = tweet_cities[0]
        else:
            city = '; '.join(tweet_cities)
    cities.append(city)
    
    tweet_states = re.findall(state_regex,tweet)
    if tweet_states:
        tweet_states = list(set(tweet_states))
        if len(tweet_states) == 1:
            state = tweet_states[0]
        else:
            state = '; '.join(tweet_states)
    states.append(state)
    
    tweet_legislators = re.findall(leg_name_regex,tweet)
    if tweet_legislators:
        if len(tweet_legislators) == 1:
            legislator = tweet_legislators[0]
        else:
            legislator = '; '.join(tweet_legislators)
    legislators.append(legislator)
    
    tweet_leg_handles = re.findall(leg_twitter_regex,tweet)
    if tweet_leg_handles:
        if len(tweet_leg_handles) == 1:
            legislator_handle = tweet_leg_handles[0]
        else:
            legislator_handle = '; '.join(tweet_leg_handles)
    legislator_handles.append(legislator_handle)
    
    if phonenumbers.PhoneNumberMatcher(tweet, "US"):
        for i,match in enumerate(phonenumbers.PhoneNumberMatcher(tweet, "US")):
            if i == 0:  
                phone_number = phonenumbers.format_number(match.number,
                                                            phonenumbers.PhoneNumberFormat.NATIONAL)
    phone_numbers.append(phone_number)
    
    urls = ''
    tweet_urls = re.findall(web_url_regex,tweet)
    if tweet_urls:
        tweet_urls = list(set(tweet_urls))
        if len(tweet_urls) == 1:
            urls = tweet_urls[0]
        else:
            urls = '; '.join(tweet_urls)
    all_urls.append(urls)
    
    
    title = tweet
    if tweet_urls:
        for urly in tweet_urls:
            title = title.replace(urly,'')
    titles.append(title)


In [10]:
city_states = []

for i, city in enumerate(cities):
    city_state = ''
    state = states[i]
    if city != '':
        if state == '':
            city_state = city
        elif state != '':
            city_state = '; '.join(list(set([cit + ', ' + state.split('; ')[0] for cit in city.split('; ')])))
        else:
            city_state = city
    else:
        if state == city:
            city_state = city
        elif state != '':
            city_state = state
            
    if city_state == 'Washington, Washington':
        city_state = 'Washington, DC'
    if city_state == 'Mobile':
        city_state = ''
    city_states.append(city_state)

In [11]:
for i, city_state in enumerate(city_states[:1000]):
    print (city_state)
    print (tweet_list[i])
    print ()

California
California D.A.'s should reveal their financial support from Planned Parenthood. https://t.co/TWJ9uEF29r


Join @NRDC &amp; sign petition to defend our environment from Trump and his fossil fuel allies in Congress. https://t.co/PwbVSJzSyW


Protect Reproductive Health And Rights  #sign #petition to Stand With Planned Parenthood &gt; https://t.co/50eRJVFMgX

California
California girls and boys are the best... boycott the rest https://t.co/CHc45cjTaR

Washington, DC
Washington Post worried about DC environment.... Let them donate or otherwise volunteer to remedy the situation instead of pointing fingers


Let's go people sign petition for this Union to put Trump out.Because the rules of the election was broken. Which was voting equality rights

Sacramento; Los Angeles
Los Angeles Times Protests erupt at Sacramento town hall meeting as ICE director answers… https://t.co/RruNs5a2av


Worth a listen to at 26:30 a Holocaust survivor setting straight the ICE director &amp; the loc

In [12]:
#### ACTIONS SEND TO SQL

prepped_to_sql = pd.DataFrame(
                        {'id': id_list,
                         'title': [title.encode('utf-8') for title in titles],
                         'description': [tweet.encode('utf-8') for tweet in tweet_list],
                         'action': [action.encode('utf-8') for action in action_list],
                         'issue': [issue.encode('utf-8') for issue in issue_list],
                         'total_score':total_score_list,
                         'relevance_score':es_score_list,
                         'legislators': legislators,
                         'legislator_twitter': legislator_handles,
                         'city':cities,
                         'state':states,
                         'city_state':city_states,
                         'phone_number': phone_numbers,
                         'urls': [url.encode('utf-8') for url in all_urls],
                         'date_of_action': dates,
                         'announced_date': tweet_timestamp_list,
                         'query_date':query_timestamp_list
                        })

prepped_to_sql.to_sql('rzst_action',conn,if_exists='replace',index=False)

In [22]:
import sunlight
from sunlight import congress
import pandas as pd

sunlight.config.API_KEY = 'thisisakey'
legislators = congress.all_legislators_in_office()

In [15]:
all_keys = set().union(*(d.keys() for d in legislators))
print(all_keys)

{'votesmart_id', 'fax', 'state_rank', 'ocd_id', 'website', 'facebook_id', 'icpsr_id', 'govtrack_id', 'first_name', 'bioguide_id', 'district', 'in_office', 'last_name', 'senate_class', 'term_start', 'fec_ids', 'youtube_id', 'state', 'chamber', 'birthday', 'title', 'crp_id', 'gender', 'middle_name', 'leadership_role', 'lis_id', 'other_names', 'term_end', 'party', 'office', 'thomas_id', 'phone', 'twitter_id', 'nickname', 'state_name', 'oc_email', 'contact_form', 'name_suffix'}


In [33]:
legislator_df = pd.DataFrame(legislators)
final_legislator_df = legislator_df[['title','first_name','last_name','state','state_name','party','website',
                                     'phone','oc_email','contact_form']]
final_legislator_df.sort_values(['state_name'], ascending=[ True],inplace=True)

/Users/brosskatz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
final_legislator_df.to_sql('rzst_legislators',conn,if_exists='replace',index=False)

In [54]:
townhalls = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/Townhalls.csv')

In [58]:
townhalls.columns = map(str.lower, townhalls.columns)
townhalls.columns

Index(['event_title', 'event_description', 'event_location', 'event_date',
       'event_time', 'event_url', 'event_related_state', 'event_legislator',
       'event_meeting_type'],
      dtype='object')

In [56]:
townhalls['event_title'] = townhalls['event_legislator'] + ' ' + townhalls['event_meeting_type']

In [57]:
townhalls = townhalls[['event_title',
                       'event_description',
                       'event_location',
                       'event_date',
                       'event_time',
                       'event_url',
                       'event_related_state',
                       'event_legislator',
                       'event_meeting_type'
                      ]]

In [59]:
townhalls.to_sql('rzst_events',conn,if_exists='replace',index=False)

In [3]:
cities = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'Top5000Population.csv'))
cities['full_city'] = cities['city'] + ', ' + cities['state'] 
print cities[:10]
city_list = list(cities['full_city'])
print city_list[:10]
# print city_list.index('Jersey City, NJ')/

searches = [city + ' indivisible' for city in city_list[35:]]
print searches[:10]

            city state population          full_city
0      New York     NY  8,363,710      New York , NY
1   Los Angeles     CA  3,833,995   Los Angeles , CA
2       Chicago     IL  2,853,114       Chicago , IL
3       Houston     TX  2,242,193       Houston , TX
4       Phoenix     AZ  1,567,924       Phoenix , AZ
5  Philadelphia     PA  1,447,395  Philadelphia , PA
6   San Antonio     TX  1,351,305   San Antonio , TX
7        Dallas     TX  1,279,910        Dallas , TX
8     San Diego     CA  1,279,329     San Diego , CA
9      San Jose     CA    948,279      San Jose , CA
['New York , NY', 'Los Angeles , CA', 'Chicago , IL', 'Houston , TX', 'Phoenix , AZ', 'Philadelphia , PA', 'San Antonio , TX', 'Dallas , TX', 'San Diego , CA', 'San Jose , CA']
['Sacramento , CA indivisible', 'Long Beach , CA indivisible', 'Mesa , AZ indivisible', 'Kansas City , MO indivisible', 'Omaha , NE indivisible', 'Cleveland , OH indivisible', 'Virginia Beach , VA indivisible', 'Miami , FL indivisible', 'Oa

In [9]:

num_page = 1
# nresults = 0
# today = datetime.datetime.today()
# weeks_ago = today - datetime.timedelta(days=14)

indivisible_list = []

for search in searches:
    try:
        search_results = google.search(search, num_page)
        for result in search_results:
            new_record = {
            'name' : result.name,  # The title of the link
            'link' : result.link, # The external link
            'google_link': result.google_link, # The google link
            'description': result.description, # The description of the link
            'thumbnail': result.thumb, # The link to a thumbnail of the website (NOT implemented yet)
            'search': search
          }
            indivisible_list.append(new_record)
    except:
        break


Error accessing: http://www.google.com/search?q=Lancaster+%2C+CA+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Fort+Collins+%2C+CO+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Warren+%2C+MI+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Hartford+%2C+CT+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=McKinney+%2C+TX+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Visalia+%2C+CA+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Killeen+%2C+TX+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Evansville+%2C+IN+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Vallejo+%2C+CA+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Ann+Arbor+%2C+MI+indivisible&start=0&num=10&nl=en
Error accessing: http://www

In [10]:
indivisible_df = pd.DataFrame(indivisible_list)
mask = ((indivisible_df['link'].str.contains("facebook.com/")) | (indivisible_df['link'].str.contains("meetup.com/")) | (indivisible_df['link'].str.contains("//indivisible")) | (indivisible_df['link'].str.contains("//www.indivisible"))) & ~(indivisible_df['link'].str.contains("indivisibleguide.com"))
indivisible_orgs = indivisible_df[mask]
print (indivisible_orgs[:30])
indivisible_orgs.to_csv('~/w210_imwithdata/imwithdata/data/static_data/indivisible_orgs_search_results2.csv',encoding='utf-8')


                                          description  \
0   Hosted by Sacramento Indivisible Regional Acti...   
10  Indivisible Long Beach CA. 545 likes · 54 talk...   
11  Indivisible Long Beach CA. 528 likes · 204 tal...   
12  Come on out early Saturday afternoon to strate...   
13  Find Meetups in Long Beach, California about M...   
14  Jan 20, 2017 ... I'm in Lakewood and I'm a mem...   
15  Indivisible Orange County is a group for those...   
17  We intend to send a message and a reminder to ...   
18  ... (Lowenthal) https://www.facebook.com/group...   
20  AZ Act Now is providing local action alerts by...   
25  Indivisible Surprise Citizens Health Care Foru...   
26  Mar 26, 2017 ... Indivisible Phx mtg/'Becoming...   
27  ... VA · INDIVISIBLE North San Diego County 1,...   
30  Welcome to Indivisible Kansas City ... Greater...   
36  Indivisible KC Week of March 27th - Missouri M...   
37  Feb 22, 2017 ... Indivisible KC, in partnershi...   
42  Indivisible Omaha works to 